Function to Load Dataset

In [2]:
def load_train():
  data = []
  labels = []

# grab the image paths and randomly shuffle them
  print("[INFO] loading images...")
  imagePaths_list = []
  imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
  #print(os.listdir(imagePaths))
  counter1 = 0
  for fileName in os.listdir(imagePaths):
    imagePaths_list.append(os.path.join(imagePaths, fileName))
	  #counter1 = counter1 + 1
	#if counter1 == 500:
		#break
  imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
  counter2 = 0
  for fileName in os.listdir(imagePaths):
    imagePaths_list.append(os.path.join(imagePaths, fileName))
    counter2 = counter2 + 1
	#if counter2 == 500:
		#break
  print(len(imagePaths_list))
  print(imagePaths_list[0])
  #print(imagePaths_list)
  random.seed(42)
  random.shuffle(imagePaths_list)

# loop over the input images
  counter = 1
  for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
    var_name = imagePath.split(os.path.sep)[-1]
	#print(counter)
	#print(var_name)
    image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = np.array(image)
    data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    counter = counter + 1
# scale the raw pixel intensities to the range [0, 1]
  data = np.array(data, dtype="float")
#print(labels)
  labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
  lb = LabelBinarizer()
  labels = lb.fit_transform(labels)
  return data, labels

In [ ]:
def sfdi_val():
  imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
  imagePaths_list_test = []
  for fileName in os.listdir(imagePaths):
    imagePaths_list_test.append(os.path.join(imagePaths, fileName))

  imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
  for fileName in os.listdir(imagePaths):
    imagePaths_list_test.append(os.path.join(imagePaths, fileName))
  test_data = []
  test_labels = []
  imagePaths_list_test.sort(key=lambda s: int(re.search(r'(\d+)\.', s).groups()[0]))
  print(imagePaths_list_test)
  random.seed(42)
  random.shuffle(imagePaths_list_test)
  #print(imagePaths_list_test[39])

  for imagePath in imagePaths_list_test:
	# load the image, pre-process it, and store it in the data list
    print(imagePath)
    var_name = imagePath.split(os.path.sep)[-1]
    if(int(var_name[5:7]) < 10):
      fin_name = var_name[0:5] + '_burn_' + var_name[6]
    else:
      fin_name = var_name[0:5] + '_burn_' + var_name[5:7]
    image = scipy.io.loadmat(imagePath)[fin_name]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = np.array(image)
    test_data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
    label = imagePath.split(os.path.sep)[-2]
    test_labels.append(label)
  test_data = np.array(test_data, dtype="float") 
#print(labels)
  test_labels = np.array(test_labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
  lb = LabelBinarizer()
  test_labels = lb.fit_transform(test_labels)
  #print(test_labels[36])
  return test_data, test_labels

The following code segment loads the input images into the data and labels arrays, and defines and compiles the normal model architecture.

In [ ]:
from sys import breakpointhook
#Normal Neural Net Architecture, Train Test split, image augmentation, Class Weightage, Test-Time Augmentation

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
import scipy.io

import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight


# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required=True,help="path to input dataset (i.e., directory of images)")
#ap.add_argument("-m", "--model", required=True,help="path to output model")
#ap.add_argument("-l", "--labelbin", required=True,help="path to output label binarizer")
#ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output accuracy/loss plot")
#args = vars(ap.parse_args())

# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 100
INIT_LR = 0.01
BS = 80
IMAGE_DIMS = (211, 211, 8)

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths_list = []
imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
#print(os.listdir(imagePaths))
counter1 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter1 = counter1 + 1
	#if counter1 == 500:
		#break
imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
counter2 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter2 = counter2 + 1
	#if counter2 == 500:
		#break
print(len(imagePaths_list))
print(imagePaths_list[0])
#print(imagePaths_list)
random.seed(42)
random.shuffle(imagePaths_list)

# loop over the input images
counter = 1
for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	var_name = imagePath.split(os.path.sep)[-1]
	print(counter)
	print(var_name)
	image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = np.array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	counter = counter + 1
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") 
#print(labels)
labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

#(trainX, testX, trainY, testY) = train_test_split(training_data,
#	training_labels, test_size=0.2, random_state=42)

# initialize the model
print("[INFO] compiling model...")
#model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
model = Sequential() 
#Conv layer: 16 3x3 kernels with (211,211,16 output)
#ReLu layer
#batch normalization 
#conv layer: 16 3x3 kernels with (211, 211, 16 output)
#ReLU layer
#batch normalization
#pooling: 2x2 pool size with 1x1 stride: output shape (211, 211, 16)
#dropout layer
#FC: 32
#ReLU
#batch normalization
#dropout
#FC: 2
#softmax: 2

#NOTE: repeat again if needed; the above architecture is shallow 
model.add(Conv2D(16, (3, 3), padding="same", input_shape=IMAGE_DIMS))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

#print(np.unique(labels.transpose()))
#print(labels.transpose()[0])
#classWeight = compute_class_weight('balanced', classes = np.unique(labels.transpose()), y = labels.transpose()[0]) 
#classWeight = dict(enumerate(classWeight))

# train the network
#print("[INFO] training network...")
#H = model.fit(
	#aug.flow(trainX, trainY, batch_size = BS),
	#validation_data = aug.flow(testX, testY),
	#steps_per_epoch=(len(trainX)//BS),
	#epochs=EPOCHS, verbose=1) #class_weight=classWeight)

# save the model to disk
#print("[INFO] serializing network...")
#model.save("burnclassifiersfdi.model")

# save the label binarizer to disk
#print("[INFO] serializing label binarizer...")
#f = open("lbsfdi.pickle", "wb")
#f.write(pickle.dumps(lb))
#f.close()

#print(H.history.keys())
#print(H.history)
#matplotlib.use("TkAgg")
# summarize history for accuracy
#plt.plot(H.history['accuracy'])
#plt.plot(H.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(H.history['loss'])
#plt.plot(H.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()

#Validation Set Results
#results = model.evaluate(testX, testY)
#print(results)


Streaming output truncated to the last 5000 lines.
Generated Image 5363.mat
1502
Generated Image 1151.mat
1503
Generated Image 5074.mat
1504
Generated Image 5541.mat
1505
Generated Image 111.mat
1506
Generated Image 5303.mat
1507
Generated Image 1476.mat
1508
Generated Image 5251.mat
1509
Generated Image 1817.mat
1510
Generated Image 4422.mat
1511
Generated Image 942.mat
1512
Generated Image 1035.mat
1513
Generated Image 1875.mat
1514
Generated Image 651.mat
1515
Generated Image 4188.mat
1516
Generated Image 1312.mat
1517
Generated Image 320.mat
1518
Generated Image 4798.mat
1519
Generated Image 4975.mat
1520
Generated Image 4892.mat
1521
Generated Image 813.mat
1522
Generated Image 1991.mat
1523
Generated Image 4031.mat
1524
Generated Image 943.mat
1525
Generated Image 4868.mat
1526
Generated Image 1660.mat
1527
Generated Image 4853.mat
1528
Generated Image 1811.mat
1529
Generated Image 5490.mat
1530
Generated Image 4151.mat
1531
Generated Image 4411.mat
1532
Generated Image 1173.mat


The following code segment runs model.fit on the model and training/testing data defined in the previous section. Note: includes the final test set output as the 80 raw images.

In [ ]:
from tensorflow.keras.optimizers import Adam
# Load the raw images as test
imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
imagePaths_list_test = []
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))

imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))
test_data = []
test_labels = []
print("length of images_paths_test " + str(imagePaths_list_test))
random.seed(42)
random.shuffle(imagePaths_list_test)

for imagePath in imagePaths_list_test:
	# load the image, pre-process it, and store it in the data list
	print(imagePath)
	var_name = imagePath.split(os.path.sep)[-1]
	image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = np.array(image)
	test_data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	test_labels.append(label)
test_data = np.array(test_data, dtype="float") 
#print(labels)
test_labels = np.array(test_labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")
opt = Adam(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
model.summary()
print("[INFO] training network...")
H = model.fit(
	trainX, trainY, batch_size = BS,
	validation_data = (testX, testY),
	steps_per_epoch=(len(trainX)//BS),
	epochs=EPOCHS, verbose=1) #class_weight=classWeight)

# save the model to disk
print("[INFO] serializing network...")
model.save("burnclassifiersfdi.model")

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open("lbsfdi.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

##print(H.history.keys())
#print(H.history)
#matplotlib.use("TkAgg")
# summarize history for accuracy
#plt.plot(H.history['accuracy'])
#plt.plot(H.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
##plt.show()
## summarize history for loss
#plt.plot(H.history['loss'])
#plt.plot(H.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
##plt.show()

##Load Raw Images as Test




length of images_paths_test ['/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_5.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_3.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_7.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_12.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_10.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_16.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_6_burn_2.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_6_burn_4.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_6_burn_6.mat', '/con

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] training network...
Epoch 1/100
40/40 [==============================] - 16s 122ms/step - loss: 0.6104 - accuracy: 0.7178 - val_loss: 29.5299 - val_accuracy: 0.4913
Epoch 2/100
40/40 [==============================] - 3s 87ms/step - loss: 0.3963 - accuracy: 0.8216 - val_loss: 17.0747 - val_accuracy: 0.4913
Epoch 3/100
40/40 [==============================] - 3s 87ms/step - loss: 0.3350 - accuracy: 0.8547 - val_loss: 7.0399 - val_accuracy: 0.5088
Epoch 4/100
40/40 [==============================] - 3s 87ms/step - loss: 0.2418 - accuracy: 0.9034 - val_loss: 11.5602 - val_accuracy: 0.4913
Epoch 5/100
40/40 [==============================] - 3s 87ms/step - loss: 0.1733 - accuracy: 0.9284 - val_loss: 2.8590 - val_accuracy: 0.5337
Epoch 6/100
40/40 [==============================] - 3s 87ms/step - loss: 0.1174 - accuracy: 0.9538 - val_loss: 17.3647 - val_accuracy: 0.4913
Epoch 7/100
40/40 [==============================] - 3s 85ms/step - loss: 0.1135 - accuracy: 0.9559 - val_loss: 4.9

ImportError: ignored

In [ ]:
#Test Set Results
results = model.evaluate(test_data, test_labels)
print(results)

3/3 [==============================] - 0s 43ms/step - loss: 4.9415 - accuracy: 0.7250
[4.941519260406494, 0.7250000238418579]


The following trains the base-level model on 10-fold cross-validation

In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
import os
import random
import cv2
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold

def create_model():
	# create model
	model = Sequential() 
	model.add(Conv2D(16, (3, 3), padding="same", input_shape=(211,211,8)))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(Conv2D(16, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
	model.add(Dropout(0.25))

	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(Conv2D(32, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=-1))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
	model.add(Dropout(0.25))

	model.add(Flatten())
	model.add(Dense(128))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
	model.add(Dense(1))
	model.add(Activation("sigmoid"))

	# Compile model
	model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
	return model
seed = 7
np.random.seed(seed)
opt = Adam(learning_rate=0.01)
# create model
model = KerasClassifier(model=create_model, epochs=100, batch_size=80)
# evaluate using 10-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
data, labels = load_train()
print(data.shape)
print(labels.shape)
results = cross_val_score(model, data, labels, cv=kfold, scoring= 'accuracy')
print(results)
#print(results.keys())

[INFO] loading images...
4000
/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/Generated Image 1069.mat
(4000, 211, 211, 8)
(4000, 1)
Epoch 1/100
45/45 [==============================] - 16s 76ms/step - loss: 0.6123 - accuracy: 0.7172
Epoch 2/100
45/45 [==============================] - 3s 77ms/step - loss: 0.3758 - accuracy: 0.8367
Epoch 3/100
45/45 [==============================] - 3s 76ms/step - loss: 0.3203 - accuracy: 0.8569
Epoch 4/100
45/45 [==============================] - 3s 78ms/step - loss: 0.3058 - accuracy: 0.8672
Epoch 5/100
45/45 [==============================] - 3s 76ms/step - loss: 0.2153 - accuracy: 0.9111
Epoch 6/100
45/45 [==============================] - 3s 77ms/step - loss: 0.1443 - accuracy: 0.9417
Epoch 7/100
45/45 [==============================] - 3s 76ms/step - loss: 0.1076 - accuracy: 0.9603
Epoch 8/100
45/45 [==============================] - 3s 75ms/step - loss: 0.1015 - accuracy: 0.9639
Epoch 9/100
45/45 [==

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
3 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.py", line 767, in fit
    **kwargs,
  File "/usr/local/lib/python3.7/dist-packages/scikeras/wrappers.p

In [ ]:
#Test Set Results
y_pred = model.predict(test_data).ravel()
#test_labels
print(y_pred)

NotFittedError: ignored

The following code snippet is meant as a test to see if the loadmat function works (imported from the Jupyter Notebook)

In [ ]:
import os
import scipy.io
img_path_x = "/Users/kushagrapandey/Documents/Wilson Project/pig_5_burn_5.mat"
name = img_path_x.split(os.path.sep)[-1]
test_img = scipy.io.loadmat(img_path_x)[name[:-4]]
test_img_np = np.array(test_img)
print(np.shape(test_img_np))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kushagrapandey/Documents/Wilson Project/pig_5_burn_5.mat'

The following code snippet uses K-fold cross validation and transfer learning to train the model. Includes:
loading data, defining model (resnet v50 with weights modified to account for 8 dimensional image), training model with K-fold cross validation

NOTE: NOT USED.

In [ ]:
#Transfer Learning, K-fold Cross validation, 
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold
#resnet50 = ResNet50V2(weights='imagenet', include_top=False)

#resnet50 = ResNet50V2(weights='imagenet', include_top=False)

# Get Resnet config in dictionary format
#config = resnet50.get_config()
    
# Change input shape to new dimensions
#config["layers"][0]["config"]["batch_input_shape"] = (None, img_height, img_width, input_channel)
    
# Create new model with config
#resnet_new = tf.keras.models.Model.from_config(config)

# Expand weights dimension to match new input channels
def multify_weights(kernel, out_channels):
  mean_1d = np.mean(kernel, axis=-2).reshape(kernel[:,:,-1:,:].shape)
  tiled = np.tile(mean_1d, (out_channels, 1))
  return(tiled)


# Loop through layers of both original model 
# and custom model and copy over weights 
# layer_modify refers to first convolutional layer
def copy_weights_tl(model_orig, custom_model, layer_modify):
  layer_to_modify = [layer_modify]

  conf = custom_model.get_config()
  layer_names = [conf['layers'][x]['name'] for x in range(len(conf['layers']))]

  for layer in model_orig.layers:
    if layer.name in layer_names:
      if layer.get_weights() != []:
        target_layer = custom_model.get_layer(layer.name)

        if layer.name in layer_to_modify:    
          kernels = layer.get_weights()[0]
          biases  = layer.get_weights()[1]

          kernels_extra_channel = np.concatenate((kernels,
                                                  multify_weights(kernels, 8 - 3)),
                                                  axis=-2)
                                                  
          target_layer.set_weights([kernels_extra_channel, biases])
          target_layer.trainable = False

        else:
          target_layer.set_weights(layer.get_weights())
          target_layer.trainable = False
def semantic_segmentation(input_size, img_height, img_width, input_channel, pretrained_weights=None):
    inputs = Input(input_size)

    resnet50 = ResNet50V2(weights='imagenet', include_top=False)

    config = resnet50.get_config()
    config["layers"][0]["config"]["batch_input_shape"] = (None, img_height, img_width, input_channel)
    config["layers"][2]["config"]["strides"] = (1, 1)


    resnet_custom = tf.keras.models.Model.from_config(config)
    modify_name = config["layers"][2]["config"]["name"]


    copy_weights_tl(resnet50, resnet_custom, modify_name)
    conv4 = resnet_custom(inputs)
    conv4 = tf.keras.activations.relu(conv4)
    #conv4_up = UpSampling2D(size=(16, 16), interpolation='bilinear')(conv4)

    #red = GlobalAveragePooling2D(name='red_pool')(conv4)
    #red = tf.keras.layers.Reshape((1, 1, 2048))(red)
    #red = Conv2D(128, 1, name='red_1_by_1')(red)
    #red = UpSampling2D(size=(320, 320), interpolation='bilinear', name='red_upsampling')(red)

    #yellow = AveragePooling2D(pool_size=(2,2), name='yellow_pool')(conv4)
    #yellow = Conv2D(128, 1, name='yellow_1_by_1')(yellow)
    #yellow = UpSampling2D(size=(32, 32), interpolation='bilinear', name='yellow_upsampling')(yellow)

    #blue = AveragePooling2D(pool_size=(4,4), name='blue_pool', padding="valid")(conv4)
    #blue = Conv2D(128, 1, name='blue_1_by_1')(blue)
    #blue = UpSampling2D(size=(16*4, 16*4), interpolation='bilinear', name='blue_upsampling')(blue)

    #green = AveragePooling2D(pool_size=(8,8), name='green_pool', padding="valid")(conv4)
    #green = Conv2D(128, 1, name='green_1_by_2')(green)
    #green = UpSampling2D(size=(160, 160), interpolation='bilinear', name='green_upsampling')(green)

    #merge9 = concatenate([conv4_up, red, yellow, blue, green])

    #merge9 = Conv2D(128, 3, activation="relu", padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv4 = Flatten()(conv4)
    conv4 = Dense(128)(conv4)
    conv4 = Activation("relu")(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Dense(1)(conv4)
    conv4 = Activation("sigmoid")(conv4)

    #output = Conv2D(3, 1, activation = 'softmax', padding='same')(merge9)
    

    model = Model(inputs, conv4)
    opt = SGD(lr=0.01)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    print(model.summary())
    return(model)




EPOCHS = 80
INIT_LR = 0.01
BS = 32
IMAGE_DIMS = (211, 211, 8)

# initialize the data and labels
data = []
labels = []
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths_list = []
imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
#for fileName in os.listdir(imagePaths):
    #if fileName != ".DS_Store":
        #imagePaths_list.append(os.path.join(imagePaths, fileName))#
#imagePaths = "/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
#for fileName in os.listdir(imagePaths):
   # if fileName != ".DS_Store":
   #     imagePaths_list.append(os.path.join(imagePaths, fileName))
#print(imagePaths_list)
#random.seed(42)
#random.shuffle(imagePaths_list)

# loop over the input images
#for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	#image = np.array(image)
	#data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	#label = imagePath.split(os.path.sep)[-2]
	#labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
#data = np.array(data, dtype="float") / 255.0
#print(labels)
#labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)
imagePaths_list = []
imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
#print(os.listdir(imagePaths))
counter1 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter1 = counter1 + 1
	if counter1 == 500:
		break
imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
counter2 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter2 = counter2 + 1
	if counter2 == 500:
		break
print(len(imagePaths_list))
print(imagePaths_list[0])
#print(imagePaths_list)
random.seed(42)
random.shuffle(imagePaths_list)

# loop over the input images
counter = 1
for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	var_name = imagePath.split(os.path.sep)[-1]
	print(counter)
	print(var_name)
	image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = np.array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	counter = counter + 1
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") 
#print(labels)
labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
def train_model_naive_split():
  all_history = {}

  inp_train_gen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=260,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
  )

  #train_data = pd.read_csv('./data/merged_data.csv')
  #train_data['label'] = train_data['label'].astype(str)
  #Y = train_data[['label']]

  skf = StratifiedKFold(n_splits = 8, random_state = 7, shuffle = True) 

  fold = 1
  for train_index, val_index in skf.split(data, labels):
    print(train_index)
    print(val_index)
    training_data = []
    training_labels = []
    validation_data = []
    validation_labels = []
    for i in train_index:
        training_data.append(data[i])
        training_labels.append(labels[i])
    for i in val_index:
        validation_data.append(data[i])
        validation_labels.append(labels[i])
    training_data = np.array(training_data)
    training_labels = np.array(training_labels)
    validation_data = np.array(validation_data)
    validation_labels = np.array(validation_labels)
    
    train_iterator = inp_train_gen.flow(training_data, training_labels, batch_size = 32)

    validation_iterator = inp_train_gen.flow(validation_data, validation_labels, batch_size = 32)

    segment = semantic_segmentation(input_size=(211,211,8), img_height = 211, img_width = 211, input_channel = 8)

    model_name = f'./output/models/best-model-kfold-{fold}.hdf5'
    history = segment.fit(train_iterator,
                        validation_data=validation_iterator,
                        epochs=80)

    all_history[f'history-fold-{fold}'] = history

    fold += 1

  return all_history

# train the network
output_data = train_model_naive_split()


# save the model to disk
print("[INFO] serializing network...")
#segment.save("burnclassifiersfdi.model")

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
#f = open("lbsfdi.pickle", "wb")
#f.write(pickle.dumps(lb))
#f.close()

#print(H.history.keys())
#print(H.history)
#matplotlib.use("TkAgg")
# summarize history for accuracy
#plt.plot(H.history['accuracy'])
#plt.plot(H.history['val_accuracy'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(H.history['loss'])
#plt.plot(H.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
#plt.show()
    
#Validation Set Results
#results = model.evaluate(testX, testY)
#print(results)



[INFO] loading images...
80
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_5.mat
1
pig_6_burn_7.mat
2
pig_8_burn_4.mat
3
pig_8_burn_3.mat
4
pig_6_burn_11.mat
5
pig_9_burn_1.mat
6
pig_7_burn_2.mat
7
pig_6_burn_6.mat
8
pig_6_burn_5.mat
9
pig_7_burn_8.mat
10
pig_5_burn_9.mat
11
pig_7_burn_10.mat
12
pig_9_burn_5.mat
13
pig_7_burn_11.mat
14
pig_5_burn_6.mat
15
pig_5_burn_11.mat
16
pig_8_burn_5.mat
17
pig_8_burn_13.mat
18
pig_7_burn_3.mat
19
pig_9_burn_10.mat
20
pig_5_burn_3.mat
21
pig_9_burn_4.mat
22
pig_7_burn_1.mat
23
pig_7_burn_14.mat
24
pig_8_burn_11.mat
25
pig_7_burn_12.mat
26
pig_7_burn_16.mat
27
pig_5_burn_15.mat
28
pig_6_burn_8.mat
29
pig_9_burn_15.mat
30
pig_8_burn_2.mat
31
pig_9_burn_9.mat
32
pig_6_burn_4.mat
33
pig_8_burn_9.mat
34
pig_5_burn_7.mat
35
pig_7_burn_6.mat
36
pig_5_burn_1.mat
37
pig_8_burn_6.mat
38
pig_5_burn_13.mat
39
pig_8_burn_7.mat
40
pig_5_burn_16.mat
41
pig_6_burn_2.mat
42
pig_7_burn_15.mat
43
pig_9_burn_6.mat
44

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (70, 211, 211, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (10, 211, 211, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


94683136/94668760 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 211, 211, 8)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 14, 14, 2048)      23580480  
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 14, 14, 2048)      0         
                                                                 
 flatten (Flatten)           (None, 401408)            0         
                                                                 
 dense (Dense)               (None, 128)               51380352  
                                                                 
 activation (Activation)     (None, 128)               0         
                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


None
Epoch 1/80
3/3 [==============================] - 22s 2s/step - loss: 1.1625 - accuracy: 0.5000 - val_loss: 1.3443 - val_accuracy: 0.5000
Epoch 2/80
3/3 [==============================] - 2s 1s/step - loss: 0.8787 - accuracy: 0.6429 - val_loss: 13.7143 - val_accuracy: 0.6000
Epoch 3/80
3/3 [==============================] - 2s 719ms/step - loss: 0.7294 - accuracy: 0.6714 - val_loss: 32.4600 - val_accuracy: 0.4000
Epoch 4/80
3/3 [==============================] - 2s 651ms/step - loss: 1.0330 - accuracy: 0.4429 - val_loss: 17.5764 - val_accuracy: 0.4000
Epoch 5/80
3/3 [==============================] - 2s 643ms/step - loss: 0.7261 - accuracy: 0.5714 - val_loss: 17.5072 - val_accuracy: 0.4000
Epoch 6/80
3/3 [==============================] - 3s 738ms/step - loss: 0.8026 - accuracy: 0.4714 - val_loss: 17.5560 - val_accuracy: 0.4000
Epoch 7/80
3/3 [==============================] - 2s 1s/step - loss: 0.8789 - accuracy: 0.5000 - val_loss: 19.5712 - val_accuracy: 0.4000
Epoch 8/80
3/3 [=

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold

def multify_weights(kernel, out_channels):
  mean_1d = np.mean(kernel, axis=-2).reshape(kernel[:,:,-1:,:].shape)
  tiled = np.tile(mean_1d, (out_channels, 1))
  return(tiled)


# Loop through layers of both original model 
# and custom model and copy over weights 
# layer_modify refers to first convolutional layer
def copy_weights_tl(model_orig, custom_model, layer_modify):
  layer_to_modify = [layer_modify]

  conf = custom_model.get_config()
  layer_names = [conf['layers'][x]['name'] for x in range(len(conf['layers']))]

  for layer in model_orig.layers:
    if layer.name in layer_names:
      if layer.get_weights() != []:
        target_layer = custom_model.get_layer(layer.name)

        if layer.name in layer_to_modify:    
          kernels = layer.get_weights()[0]
          biases  = layer.get_weights()[1]

          kernels_extra_channel = np.concatenate((kernels,
                                                  multify_weights(kernels, 8 - 3)),
                                                  axis=-2)
                                                  
          target_layer.set_weights([kernels_extra_channel, biases])
          target_layer.trainable = False

        else:
          target_layer.set_weights(layer.get_weights())
          target_layer.trainable = False
def semantic_segmentation(input_size, img_height, img_width, input_channel, pretrained_weights=None):
    inputs = Input(input_size)

    resnet50 = ResNet50V2(weights='imagenet', include_top=False)

    config = resnet50.get_config()
    config["layers"][0]["config"]["batch_input_shape"] = (None, img_height, img_width, input_channel)
    config["layers"][2]["config"]["strides"] = (1, 1)


    resnet_custom = tf.keras.models.Model.from_config(config)
    modify_name = config["layers"][2]["config"]["name"]


    copy_weights_tl(resnet50, resnet_custom, modify_name)
    conv4 = resnet_custom(inputs)
    conv4 = tf.keras.activations.relu(conv4)
    #conv4_up = UpSampling2D(size=(16, 16), interpolation='bilinear')(conv4)

    #red = GlobalAveragePooling2D(name='red_pool')(conv4)
    #red = tf.keras.layers.Reshape((1, 1, 2048))(red)
    #red = Conv2D(128, 1, name='red_1_by_1')(red)
    #red = UpSampling2D(size=(320, 320), interpolation='bilinear', name='red_upsampling')(red)

    #yellow = AveragePooling2D(pool_size=(2,2), name='yellow_pool')(conv4)
    #yellow = Conv2D(128, 1, name='yellow_1_by_1')(yellow)
    #yellow = UpSampling2D(size=(32, 32), interpolation='bilinear', name='yellow_upsampling')(yellow)

    #blue = AveragePooling2D(pool_size=(4,4), name='blue_pool', padding="valid")(conv4)
    #blue = Conv2D(128, 1, name='blue_1_by_1')(blue)
    #blue = UpSampling2D(size=(16*4, 16*4), interpolation='bilinear', name='blue_upsampling')(blue)

    #green = AveragePooling2D(pool_size=(8,8), name='green_pool', padding="valid")(conv4)
    #green = Conv2D(128, 1, name='green_1_by_2')(green)
    #green = UpSampling2D(size=(160, 160), interpolation='bilinear', name='green_upsampling')(green)

    #merge9 = concatenate([conv4_up, red, yellow, blue, green])

    #merge9 = Conv2D(128, 3, activation="relu", padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv4 = Flatten()(conv4)
    conv4 = Dense(128)(conv4)
    conv4 = Activation("relu")(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Dense(1)(conv4)
    conv4 = Activation("sigmoid")(conv4)

    #output = Conv2D(3, 1, activation = 'softmax', padding='same')(merge9)
    

    model = Model(inputs, conv4)
    opt = SGD(lr=0.01)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    print(model.summary())
    return(model)
model_transfer = semantic_segmentation(input_size=(211,211,8), img_height = 211, img_width = 211, input_channel = 8)
print("[INFO] training network...")
H = model_transfer.fit(
	trainX, trainY, batch_size = BS,
	validation_data = (testX, testY),
	steps_per_epoch=(len(trainX)//BS),
	epochs=EPOCHS, verbose=1) #class_weight=classWeight)

94683136/94668760 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 211, 211, 8)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 14, 14, 2048)      23580480  
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 14, 14, 2048)      0         
                                                                 
 flatten_11 (Flatten)        (None, 401408)            0         
                                                                 
 dense_22 (Dense)            (None, 128)               51380352  
                                                                 
 activation_66 (Activation)  (None, 128)               0         
                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


None
[INFO] training network...
Epoch 1/100
40/40 [==============================] - 13s 214ms/step - loss: 0.6550 - accuracy: 0.6897 - val_loss: 1.0229 - val_accuracy: 0.6275
Epoch 2/100
40/40 [==============================] - 7s 173ms/step - loss: 0.4118 - accuracy: 0.8178 - val_loss: 0.7560 - val_accuracy: 0.6837
Epoch 3/100
40/40 [==============================] - 7s 173ms/step - loss: 0.3066 - accuracy: 0.8675 - val_loss: 0.4331 - val_accuracy: 0.8037
Epoch 4/100
40/40 [==============================] - 7s 172ms/step - loss: 0.2270 - accuracy: 0.9100 - val_loss: 0.3946 - val_accuracy: 0.8313
Epoch 5/100
40/40 [==============================] - 7s 172ms/step - loss: 0.1635 - accuracy: 0.9403 - val_loss: 0.3464 - val_accuracy: 0.8512
Epoch 6/100
40/40 [==============================] - 7s 172ms/step - loss: 0.0977 - accuracy: 0.9737 - val_loss: 0.3655 - val_accuracy: 0.8512
Epoch 7/100
40/40 [==============================] - 7s 172ms/step - loss: 0.0735 - accuracy: 0.9834 - val_lo

In [ ]:
#Test Set Results
results = model_transfer.evaluate(test_data, test_labels)
print(results)

3/3 [==============================] - 2s 233ms/step - loss: 0.0973 - accuracy: 0.9750
[0.09730099886655807, 0.9750000238418579]


In [ ]:
#model_transfer_kfold = semantic_segmentation(input_size=(211,211,8), img_height = 211, img_width = 211, input_channel = 8)
from sklearn.model_selection import cross_val_score
seed = 7
np.random.seed(seed)
# create model
model_transfer_kfold = KerasClassifier(model=semantic_segmentation(input_size=(211,211,8), img_height = 211, img_width = 211, input_channel = 8), epochs=30, batch_size=80)
# evaluate using 10-fold cross validation
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
new_results = cross_val_score(model_transfer_kfold, data, labels, cv=kfold, scoring= 'accuracy')
print(new_results)
print(new_results.keys())

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 211, 211, 8)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 14, 14, 2048)      23580480  
                                                                 
 tf.nn.relu_11 (TFOpLambda)  (None, 14, 14, 2048)      0         
                                                                 
 flatten_22 (Flatten)        (None, 401408)            0         
                                                                 
 dense_44 (Dense)            (None, 128)               51380352  
                                                                 
 activation_88 (Activation)  (None, 128)               0         
                                                                 
 batch_normalization_66 (Bat  (None, 128)              512

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


None
Epoch 1/30
45/45 [==============================] - 9s 139ms/step - loss: 0.6173 - accuracy: 0.6875
Epoch 2/30
45/45 [==============================] - 6s 136ms/step - loss: 0.3793 - accuracy: 0.8331
Epoch 3/30
45/45 [==============================] - 6s 137ms/step - loss: 0.2855 - accuracy: 0.8781
Epoch 4/30
45/45 [==============================] - 6s 137ms/step - loss: 0.2100 - accuracy: 0.9203
Epoch 5/30
45/45 [==============================] - 6s 140ms/step - loss: 0.1385 - accuracy: 0.9544
Epoch 6/30
45/45 [==============================] - 6s 136ms/step - loss: 0.1042 - accuracy: 0.9681
Epoch 7/30
45/45 [==============================] - 6s 138ms/step - loss: 0.0667 - accuracy: 0.9861
Epoch 8/30
45/45 [==============================] - 6s 137ms/step - loss: 0.0473 - accuracy: 0.9936
Epoch 9/30
45/45 [==============================] - 6s 138ms/step - loss: 0.0346 - accuracy: 0.9961
Epoch 10/30
45/45 [==============================] - 6s 138ms/step - loss: 0.0267 - accuracy: 0

AttributeError: ignored

In [ ]:
%load_ext Cython

The following defines and trains a conditional GAN over the 80 input images. NOTES: maybe train over the 8000 augmented images to get a better sense of the performance

In [ ]:
## GAN to generate new data
# example of training an conditional gan on the fashion mnist dataset

#%%cython
import numpy
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
#import pandas
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

# define the standalone discriminator model
def define_discriminator(in_shape=(211,211,8), n_classes=2):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50, input_length = 8)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = 211*211
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2))(merge) #output: (104,104,128)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(256, (8,8), strides=(4,4))(fe) #output: (24, 24, 256)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	print(model.summary())
	return model

# define the standalone generator model
def define_generator(latent_dim, n_classes=2):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 24 * 24
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((24, 24, 1))(li) #DO WE NEED 8 HERE OR 1
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 256 * 24 * 24
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((24, 24, 256))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 48x48
	gen = Conv2DTranspose(128, 3, 2, padding='same',)(merge) 
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 96x96
	gen = Conv2DTranspose(64, 3, 2, padding='same',)(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 211x211
	gen = Conv2DTranspose(64, 20, 2, padding='valid', output_padding = (1,1))(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	out_layer = Conv2D(8, (24,24), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	print(model.summary())
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	print(model.summary())
	return model

# load fashion mnist images
def load_real_samples():
	data = []
	labels = []
	imagePaths_list = []
	imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
	for fileName in os.listdir(imagePaths):
		imagePaths_list.append(os.path.join(imagePaths, fileName))
	#imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
	#for fileName in os.listdir(imagePaths):
		#imagePaths_list.append(os.path.join(imagePaths, fileName))
	print(imagePaths_list)
	random.seed(42)
	random.shuffle(imagePaths_list)

	# loop over the input images
	for imagePath in imagePaths_list:
		# load the image, pre-process it, and store it in the data list
		#print(imagePath)
		var_name = imagePath.split(os.path.sep)[-1]
		image = scipy.io.loadmat(imagePath)[var_name[:-4]]
		#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
		image = np.array(image)
		data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
		label = imagePath.split(os.path.sep)[-2]
		labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
	data = np.array(data, dtype="float")
#print(labels)
	labels = np.array(labels)
	lb = LabelBinarizer()
	labels = lb.fit_transform(labels)
	return [data, labels]

# # select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	data_output = []
	label_output = []
	for i in range(80):
		# choose random instances
		# select images and labels
		X, Y = images[i], labels[i]
		data_output.append(X)
		label_output.append(Y)
		# generate class labels
	data_output = numpy.array(data_output)
	label_output = numpy.array(label_output)
	return data_output, label_output

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=2):
	# generate points in the latent space
	x_input = randn(latent_dim * 80)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(80, latent_dim)
	# generate labels
	labels = randint(0, n_classes, (80, 1))
	return z_input, labels

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	#y = zeros((n_samples, 1))
	return images, labels_input

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=20):
	print("dataset[0].shape[0] = " + str(dataset[0].shape[0]))
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		# get randomly selected 'real' samples
		print("line 219")
		X_real, labels_real = generate_real_samples(dataset, half_batch)
		print("line 221")
		# update discriminator model weights
		print(labels_real.shape)
		d_loss1, _ = d_model.train_on_batch([X_real, labels_real], ones((80, 1)))
		print("line 225")
		# generate 'fake' examples
		X_fake, labels_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		print("line 228")
		# update discriminator model weights
		d_loss2, _ = d_model.train_on_batch([X_fake, labels_fake], ones((80, 1)))
		print("line 231")
		# prepare points in latent space as input for the generator
		z_input, labels_input = generate_latent_points(latent_dim, n_batch)
		print("line 234")
		# create inverted labels for the fake samples
		y_gan = ones((80, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
		print("line 239")
		# summarize loss on this batch
		print('>%d, d1=%.3f, d2=%.3f g=%.3f' %
			(i+1, d_loss1, d_loss2, g_loss))
	# save the generator model
	g_model.save('cgan_generator.h5')

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        100         ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1, 44521)     2270571     ['embedding[0][0]']              
                                                                                                  
 input_2 (InputLayer)           [(None, 211, 211, 8  0           []                               
                                )]                                                            

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


                                                                                                  
 reshape_1 (Reshape)            (None, 24, 24, 1)    0           ['dense_2[0][0]']                
                                                                                                  
 concatenate_1 (Concatenate)    (None, 24, 24, 257)  0           ['reshape_2[0][0]',              
                                                                  'reshape_1[0][0]']              
                                                                                                  
 conv2d_transpose (Conv2DTransp  (None, 48, 48, 128)  296192     ['concatenate_1[0][0]']          
 ose)                                                                                             
                                                                                                  
 leaky_re_lu_3 (LeakyReLU)      (None, 48, 48, 128)  0           ['conv2d_transpose[0][0]']       
          

line 239
>20, d1=0.000, d2=0.000 g=0.000


In [ ]:
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=2):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

from google.colab import files
uploaded = files.upload()
# load model
cgan_generator_model = load_model("/content/cgan_generator.h5")
# generate images
latent_points, labels = generate_latent_points(100, 4000)
# specify labels
labels = asarray([x for _ in range(2000) for x in range(2)])
# generate images
X  = cgan_generator_model.predict([latent_points, labels])
X = X*10
print(X.shape)
# plot the result


Saving cgan_generator.h5 to cgan_generator (1).h5


125/125 [==============================] - 3s 21ms/step
(4000, 211, 211, 8)


In [ ]:
import scipy.io
# create and save a plot of generated images
print(labels.shape)
X = X/10
print(X.max())
print(X.min())
def save_plot(examples, n):
	# plot images
	for i in range(4000):
		## define subplot
		#pyplot.subplot(2000, 2, 1 + i)
		## turn off axis
		#pyplot.axis('off')
		# plot raw pixel data
		#pyplot.imshow(examples[i, :, :, :])
		mdic = {"Generated Image cGAN " + str(i+1): examples[i, :, :, :]}
		#print(mdic)
		print(labels[i])
		if (labels[i] == 0):
			scipy.io.savemat("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/Generated Image " + str(i+1) + ".mat", mdic)
		#print(mdic)
		else:
			scipy.io.savemat("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4/Generated Image " + str(i+1) + ".mat", mdic)
		#scipy.io.savemat("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4/Generated Image " + str(i+1) + ".mat", mdic)
	#pyplot.show()
save_plot(X, 10)

(4000,)
0.069751635
0.0007954778
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1

In [ ]:
!pip install tensorflow_docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow_docs (from versions: none)
ERROR: No matching distribution found for tensorflow_docs


In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from sys import breakpointhook
#Normal Neural Net Architecture, Train Test split, image augmentation, Class Weightage, Test-Time Augmentation

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
import scipy.io

import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
#from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
#from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio
import re

batch_size = 80
num_channels = 8
num_classes = 1
image_size = 211
latent_dim = 100

all_digits, all_labels = load_train()
all_digits = np.reshape(all_digits, (-1, 211, 211, 8))
#all_labels = keras.utils.to_categorical(all_labels, 2)
#lb = LabelBinarizer()
#all_labels = lb.fit_transform(all_labels)

dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((211, 211, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(24 * 24 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((24, 24, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), #(80,48, 48, 128)
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"), #(80,96,96,128)
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(64, 20, 2, padding='valid', output_padding = (1,1)), #(80, 211,211,64)
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(8, (7, 7), padding="same", activation="sigmoid"), #(80, 211,211,8)
    ],
    name="generator",
)

class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data
        one_hot_labels = tf.cast(one_hot_labels, tf.float32)
        real_images = tf.cast(real_images, tf.float32)

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        
        print(one_hot_labels)
        #print(random_latent_vectors.shape)
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=50)

[INFO] loading images...
4000
/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/Generated Image 1069.mat
Shape of training images: (4000, 211, 211, 8)
Shape of training labels: (4000, 1)
101 9
Epoch 1/50
Tensor("Cast:0", shape=(None, 1), dtype=float32)
Tensor("Cast:0", shape=(None, 1), dtype=float32)
50/50 [==============================] - 54s 470ms/step - g_loss: 0.6604 - d_loss: 0.6854
Epoch 2/50
50/50 [==============================] - 24s 471ms/step - g_loss: 1.6519 - d_loss: 0.2887
Epoch 3/50
50/50 [==============================] - 24s 471ms/step - g_loss: 1.5765 - d_loss: 0.4758
Epoch 4/50
50/50 [==============================] - 24s 472ms/step - g_loss: 2.2273 - d_loss: 0.4213
Epoch 5/50
50/50 [==============================] - 24s 472ms/step - g_loss: 1.7851 - d_loss: 0.3114
Epoch 6/50
50/50 [==============================] - 24s 471ms/step - g_loss: 3.5501 - d_loss: 0.0614
Epoch 7/50
50/50 [==============================] - 24s 471m

In [ ]:
print(all_labels)

[[1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]]


In [4]:
trained_gen = cond_gan.generator

# Choose the number of intermediate images that would be generated in
# between the interpolation + 2 (start and last images).
num_interpolation = 2000  # param {type:"integer"}

# Sample noise for the interpolation.
interpolation_noise = tf.random.normal(shape=(1, latent_dim))
interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))


def interpolate_class_1_2():
    # Convert the start and end labels to one-hot encoded vectors.
    #first_label = keras.utils.to_categorical([first_number], num_classes)
    #second_label = keras.utils.to_categorical([second_number], num_classes)
    #first_label = tf.cast(first_label, tf.float32)
    #second_label = tf.cast(second_label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    #percent_second_label = tf.linspace(0, 1, num_interpolation)[:, None]
    #percent_second_label = tf.cast(percent_second_label, tf.float32)
    interpolation_labels = np.zeros((2000, 1), dtype=np.float32)

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake


start_class = 1  # param {type:"slider", min:0, max:9, step:1}
end_class = 5  # param {type:"slider", min:0, max:9, step:1}

fake_images = interpolate_class_1_2()

for i in range(2000):
  mdic = {"Generated Image " + str(i+1): fake_images[i]}
  scipy.io.savemat("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/Generated Image " + str(i+1) + ".mat", mdic)

def interpolate_class_3_4():
    interpolation_labels = np.ones((2000, 1), dtype=np.float32)

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise, interpolation_labels], 1)
    fake = trained_gen.predict(noise_and_labels)
    return fake
fake_images_2 = interpolate_class_3_4()
for i in range(2000):
  mdic = {"Generated Image " + str(i+2001): fake_images_2[i]}
  scipy.io.savemat("/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4/Generated Image " + str(i+2001) + ".mat", mdic)

63/63 [==============================] - 2s 29ms/step


In [ ]:
import tensorflow as tf
import keras
percent_second_label = tf.linspace(0, 1, 9)[:, None]
percent_second_label = tf.cast(percent_second_label, tf.float32)
print(percent_second_label)
print(1-percent_second_label)
first_label = keras.utils.to_categorical([1], 10)
first_label = tf.cast(first_label, tf.float32)
print(first_label)
print(first_label * (1 - percent_second_label))
interpolation_labels = first_label * (1 - percent_second_label)
interpolation_noise = tf.random.normal(shape=(1, 100))
interpolation_noise = tf.repeat(interpolation_noise, repeats=9)
interpolation_noise = tf.reshape(interpolation_noise, (9, 100))
print(tf.concat([interpolation_noise, interpolation_labels], 1).shape)

tf.Tensor(
[[0.   ]
 [0.125]
 [0.25 ]
 [0.375]
 [0.5  ]
 [0.625]
 [0.75 ]
 [0.875]
 [1.   ]], shape=(9, 1), dtype=float32)
tf.Tensor(
[[1.   ]
 [0.875]
 [0.75 ]
 [0.625]
 [0.5  ]
 [0.375]
 [0.25 ]
 [0.125]
 [0.   ]], shape=(9, 1), dtype=float32)
tf.Tensor([[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.875 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.75  0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.625 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.5   0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.375 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.25  0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.125 0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]], shape=(9, 10), dtype=float32)
(9, 110)


Augmentation

In [ ]:
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight


# initialize the data and labels
data_cat1 = []
labels_cat1 = []
data_cat2 = []
labels_cat2 = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths_list_cat1 = []
imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
for fileName in os.listdir(imagePaths):
    print(imagePaths + "/" + str(fileName) + " " + str(fileName[:-4]))
    image = scipy.io.loadmat(imagePaths + "/" + str(fileName))[fileName[:-4]]
    image = np.array(image)
    data_cat1.append(image)
 
	  # extract the class label from the image path and update the
	  # labels list
    label = "Histo Score 1-2"
    labels_cat1.append(label)
imagePaths2 = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
imagePaths_list_cat2 = []
for fileName in os.listdir(imagePaths2):
    print(imagePaths2 + "/" + str(fileName) + " " + str(fileName[:-4]))
    image = scipy.io.loadmat(imagePaths2 + "/" + str(fileName))[fileName[:-4]]
    image = np.array(image)
    data_cat2.append(image)
    label = "Histo Score 3-4"
    labels_cat2.append(label)
# loop over the input images
#for imagePath in imagePaths_list_cat1:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	#image = np.array(image)
	#data_cat1.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	#label = imagePath.split(os.path.sep)[-2]
	#labels_cat1.append(label)

# scale the raw pixel intensities to the range [0, 1]
data_cat1 = np.array(data_cat1, dtype="float") 
#print(labels)
labels_cat1 = np.array(labels_cat1)
print("[INFO] data matrix: {:.2f}MB".format(
	data_cat1.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels_cat1 = lb.fit_transform(labels_cat1)

#for imagePath in imagePaths_list_cat2:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	#var_name = imagePath.split(os.path.sep)[-1]
	#image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	#image = np.array(image)
	#data_cat2.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	#label = imagePath.split(os.path.sep)[-2]
#	labels_cat2.append(label)

# scale the raw pixel intensities to the range [0, 1]
data_cat2 = np.array(data_cat2, dtype="float") 
#print(labels)
labels_cat2 = np.array(labels_cat2)
print("[INFO] data matrix: {:.2f}MB".format(
	data_cat2.nbytes / (1024 * 1000.0)))

# binarize the labels
#lb = LabelBinarizer()
labels_cat2 = lb.fit_transform(labels_cat2)


aug = ImageDataGenerator(rotation_range=30,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
imageGen_cat1 = aug.flow(data_cat1)
i = 1
#CATEGORY 1-2
#for image_list in imageGen_cat1:
    #for image in image_list:
    #    mdic = {"Generated Image " + str(i): image}
     #   print(mdic)
     #   scipy.io.savemat("/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/Generated Image " + str(i) + ".mat", mdic)
     #   print(i)
     #   if i == 4000:
     #       break
      #  i = i+1
    #else:
    #    continue
   # break
aug = ImageDataGenerator(rotation_range=30,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
imageGen_cat2 = aug.flow(data_cat2)
i = 4001
for image_list in imageGen_cat2:
    for image in image_list:
        mdic = {"Generated Image " + str(i): image}
        print(mdic)
        scipy.io.savemat("/content/drive/MyDrive/Wilson/Augmented Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4/Generated Image " + str(i) + ".mat", mdic)
        if i == 8000:
            break
        i = i+1
    else:
        continue
    break

[INFO] loading images...
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_5.mat pig_5_burn_5
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_3.mat pig_5_burn_3
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_7.mat pig_5_burn_7
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_12.mat pig_5_burn_12
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_10.mat pig_5_burn_10
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_5_burn_16.mat pig_5_burn_16
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_6_burn_2.mat pig_6_burn_2
/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_6_burn_4.mat pig_6_burn_4
/content/drive/MyDrive/Wilson/Raw

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (34, 211, 211, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (46, 211, 211, 8) (8 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


Streaming output truncated to the last 5000 lines.
         0.73718584, 0.74290264],
        [0.46362776, 0.44239402, 0.5099386 , ..., 0.7310397 ,
         0.7352575 , 0.74291456],
        [0.46230906, 0.44387895, 0.5108976 , ..., 0.7331228 ,
         0.73474175, 0.7435386 ],
        ...,
        [0.52503073, 0.4976234 , 0.5660608 , ..., 0.7886132 ,
         0.78779894, 0.7975647 ],
        [0.5248769 , 0.49773258, 0.5651528 , ..., 0.78697866,
         0.78500926, 0.7953212 ],
        [0.5232413 , 0.49636254, 0.5632447 , ..., 0.78264046,
         0.7814121 , 0.7916854 ]],

       [[0.4702465 , 0.44744125, 0.5135351 , ..., 0.7300447 ,
         0.7387964 , 0.7435338 ],
        [0.4655832 , 0.4436029 , 0.51105005, ..., 0.730297  ,
         0.736241  , 0.7428887 ],
        [0.4630124 , 0.44308692, 0.5103861 , ..., 0.73201174,
         0.7350168 , 0.7432058 ],
        ...,
        [0.52387494, 0.49689332, 0.5639839 , ..., 0.7843211 ,
         0.7828057 , 0.793094  ],
        [0.5222393 , 0.

Code snippet for verification of library overwrite in for loops

In [ ]:
for i in range(10):
  dictionary = {str(i): i}
print(dictionary)

{'9': 9}


Attempting to get 25 GB of RAM 

In [ ]:
i = []
while True:
  i.append(i)

Code to mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a = 3

In [ ]:
print(a+5)

8


NOTE: cGAN Models below

In [4]:
from sys import breakpointhook
#Normal Neural Net Architecture, Train Test split, image augmentation, Class Weightage, Test-Time Augmentation

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
import scipy.io

import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight


# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", required=True,help="path to input dataset (i.e., directory of images)")
#ap.add_argument("-m", "--model", required=True,help="path to output model")
#ap.add_argument("-l", "--labelbin", required=True,help="path to output label binarizer")
#ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output accuracy/loss plot")
#args = vars(ap.parse_args())

# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 100
INIT_LR = 0.01
BS = 80
IMAGE_DIMS = (211, 211, 8)

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths_list = []
imagePaths = "/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
#print(os.listdir(imagePaths))
counter1 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter1 = counter1 + 1
	#if counter1 == 500:
		#break
imagePaths = "/content/drive/MyDrive/Wilson/cGAN Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
counter2 = 0
for fileName in os.listdir(imagePaths):
	imagePaths_list.append(os.path.join(imagePaths, fileName))
	counter2 = counter2 + 1
	#if counter2 == 500:
		#break
print(len(imagePaths_list))
print(imagePaths_list[0])
#print(imagePaths_list)
random.seed(42)
random.shuffle(imagePaths_list)

# loop over the input images
counter = 1
for imagePath in imagePaths_list:
	# load the image, pre-process it, and store it in the data list
	#print(imagePath)
	var_name = imagePath.split(os.path.sep)[-1]
	print(counter)
	print(var_name)
	image = scipy.io.loadmat(imagePath)[var_name[:-4]]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = np.array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
	counter = counter + 1
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") 
#print(labels)
labels = np.array(labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

#(trainX, testX, trainY, testY) = train_test_split(training_data,
#	training_labels, test_size=0.2, random_state=42)

# initialize the model
print("[INFO] compiling model...")
#model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0], depth=IMAGE_DIMS[2], classes=len(lb.classes_))
model = Sequential() 
#Conv layer: 16 3x3 kernels with (211,211,16 output)
#ReLu layer
#batch normalization 
#conv layer: 16 3x3 kernels with (211, 211, 16 output)
#ReLU layer
#batch normalization
#pooling: 2x2 pool size with 1x1 stride: output shape (211, 211, 16)
#dropout layer
#FC: 32
#ReLU
#batch normalization
#dropout
#FC: 2
#softmax: 2

#NOTE: repeat again if needed; the above architecture is shallow 
model.add(Conv2D(16, (3, 3), padding="same", input_shape=IMAGE_DIMS))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

Streaming output truncated to the last 5000 lines.
Generated Image 3363.mat
1502
Generated Image 1083.mat
1503
Generated Image 3074.mat
1504
Generated Image 3541.mat
1505
Generated Image 43.mat
1506
Generated Image 3303.mat
1507
Generated Image 1408.mat
1508
Generated Image 3251.mat
1509
Generated Image 1749.mat
1510
Generated Image 2422.mat
1511
Generated Image 874.mat
1512
Generated Image 967.mat
1513
Generated Image 1807.mat
1514
Generated Image 583.mat
1515
Generated Image 2188.mat
1516
Generated Image 1244.mat
1517
Generated Image 252.mat
1518
Generated Image 2798.mat
1519
Generated Image 2975.mat
1520
Generated Image 2892.mat
1521
Generated Image 745.mat
1522
Generated Image 1923.mat
1523
Generated Image 2031.mat
1524
Generated Image 875.mat
1525
Generated Image 2868.mat
1526
Generated Image 1592.mat
1527
Generated Image 2853.mat
1528
Generated Image 1743.mat
1529
Generated Image 3490.mat
1530
Generated Image 2151.mat
1531
Generated Image 2411.mat
1532
Generated Image 1105.mat
15

In [5]:
from tensorflow.keras.optimizers import Adam, SGD
# Load the raw images as test


#aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	#height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	#horizontal_flip=True, fill_mode="nearest")
opt = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
model.summary()
print("[INFO] training network...")
H = model.fit(
	trainX, trainY, batch_size = BS,
	validation_data = (testX, testY),
	steps_per_epoch=(len(trainX)//BS),
	epochs=EPOCHS, verbose=1) #class_weight=classWeight)

# save the model to disk
#print("[INFO] serializing network...")
#model.save("burnclassifiersfdi.model")

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
#f = open("lbsfdi.pickle", "wb")
#f.write(pickle.dumps(lb))
#f.close()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 211, 211, 16)      1168      
                                                                 
 activation (Activation)     (None, 211, 211, 16)      0         
                                                                 
 batch_normalization (BatchN  (None, 211, 211, 16)     64        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 211, 211, 16)      2320      
                                                                 
 activation_1 (Activation)   (None, 211, 211, 16)      0         
                                                                 
 batch_normalization_1 (Batc  (None, 211, 211, 16)     64        
 hNormalization)                                        

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


[INFO] training network...
Epoch 1/100
40/40 [==============================] - 18s 132ms/step - loss: 0.3311 - accuracy: 0.8631 - val_loss: 2.6810 - val_accuracy: 0.5088
Epoch 2/100
40/40 [==============================] - 3s 86ms/step - loss: 0.0428 - accuracy: 0.9928 - val_loss: 5.9722 - val_accuracy: 0.5088
Epoch 3/100
40/40 [==============================] - 3s 86ms/step - loss: 0.0195 - accuracy: 0.9969 - val_loss: 8.0891 - val_accuracy: 0.5088
Epoch 4/100
40/40 [==============================] - 3s 86ms/step - loss: 0.0154 - accuracy: 0.9978 - val_loss: 10.0732 - val_accuracy: 0.5088
Epoch 5/100
40/40 [==============================] - 3s 85ms/step - loss: 0.0118 - accuracy: 0.9984 - val_loss: 11.7669 - val_accuracy: 0.5088
Epoch 6/100
40/40 [==============================] - 3s 88ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 12.4748 - val_accuracy: 0.5088
Epoch 7/100
40/40 [==============================] - 3s 85ms/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 12.4

In [16]:
imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
imagePaths_list_test = []
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))

imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))
test_data = []
test_labels = []
print("length of images_paths_test " + str(imagePaths_list_test))
random.seed(42)
random.shuffle(imagePaths_list_test)

for imagePath in imagePaths_list_test:
	# load the image, pre-process it, and store it in the data list
	print(imagePath)
	var_name = imagePath.split(os.path.sep)[-1]
	if(int(var_name[5:7]) < 10):
		fin_name = var_name[0:5] + '_burn_' + var_name[6]
	else:
		fin_name = var_name[0:5] + '_burn_' + var_name[5:7]
	image = scipy.io.loadmat(imagePath)[fin_name]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = np.array(image)
	test_data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	test_labels.append(label)
test_data = np.array(test_data, dtype="float") 
#print(labels)
test_labels = np.array(test_labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)
results = model.evaluate(test_data, test_labels)
print(results)

length of images_paths_test ['/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_503.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_505.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_507.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_510.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_512.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_516.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_602.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_604.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_606.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo S

In [9]:
from tensorflow.keras.applications import ResNet50V2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout, Input
import matplotlib
matplotlib.use("Agg")
import scipy.io
import glob
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
#from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Activation, Flatten, Dropout
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os
#from tk import *
from sklearn.utils import compute_class_weight
from sklearn.model_selection import StratifiedKFold

def multify_weights(kernel, out_channels):
  mean_1d = np.mean(kernel, axis=-2).reshape(kernel[:,:,-1:,:].shape)
  tiled = np.tile(mean_1d, (out_channels, 1))
  return(tiled)


# Loop through layers of both original model 
# and custom model and copy over weights 
# layer_modify refers to first convolutional layer
def copy_weights_tl(model_orig, custom_model, layer_modify):
  layer_to_modify = [layer_modify]

  conf = custom_model.get_config()
  layer_names = [conf['layers'][x]['name'] for x in range(len(conf['layers']))]

  for layer in model_orig.layers:
    if layer.name in layer_names:
      if layer.get_weights() != []:
        target_layer = custom_model.get_layer(layer.name)

        if layer.name in layer_to_modify:    
          kernels = layer.get_weights()[0]
          biases  = layer.get_weights()[1]

          kernels_extra_channel = np.concatenate((kernels,
                                                  multify_weights(kernels, 8 - 3)),
                                                  axis=-2)
                                                  
          target_layer.set_weights([kernels_extra_channel, biases])
          target_layer.trainable = False

        else:
          target_layer.set_weights(layer.get_weights())
          target_layer.trainable = False
def semantic_segmentation(input_size, img_height, img_width, input_channel, pretrained_weights=None):
    inputs = Input(input_size)

    resnet50 = ResNet50V2(weights='imagenet', include_top=False)

    config = resnet50.get_config()
    config["layers"][0]["config"]["batch_input_shape"] = (None, img_height, img_width, input_channel)
    config["layers"][2]["config"]["strides"] = (1, 1)


    resnet_custom = tf.keras.models.Model.from_config(config)
    modify_name = config["layers"][2]["config"]["name"]


    copy_weights_tl(resnet50, resnet_custom, modify_name)
    conv4 = resnet_custom(inputs)
    conv4 = tf.keras.activations.relu(conv4)
    #conv4_up = UpSampling2D(size=(16, 16), interpolation='bilinear')(conv4)

    #red = GlobalAveragePooling2D(name='red_pool')(conv4)
    #red = tf.keras.layers.Reshape((1, 1, 2048))(red)
    #red = Conv2D(128, 1, name='red_1_by_1')(red)
    #red = UpSampling2D(size=(320, 320), interpolation='bilinear', name='red_upsampling')(red)

    #yellow = AveragePooling2D(pool_size=(2,2), name='yellow_pool')(conv4)
    #yellow = Conv2D(128, 1, name='yellow_1_by_1')(yellow)
    #yellow = UpSampling2D(size=(32, 32), interpolation='bilinear', name='yellow_upsampling')(yellow)

    #blue = AveragePooling2D(pool_size=(4,4), name='blue_pool', padding="valid")(conv4)
    #blue = Conv2D(128, 1, name='blue_1_by_1')(blue)
    #blue = UpSampling2D(size=(16*4, 16*4), interpolation='bilinear', name='blue_upsampling')(blue)

    #green = AveragePooling2D(pool_size=(8,8), name='green_pool', padding="valid")(conv4)
    #green = Conv2D(128, 1, name='green_1_by_2')(green)
    #green = UpSampling2D(size=(160, 160), interpolation='bilinear', name='green_upsampling')(green)

    #merge9 = concatenate([conv4_up, red, yellow, blue, green])

    #merge9 = Conv2D(128, 3, activation="relu", padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv4 = Flatten()(conv4)
    conv4 = Dense(128)(conv4)
    conv4 = Activation("relu")(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)
    conv4 = Dense(1)(conv4)
    conv4 = Activation("sigmoid")(conv4)

    #output = Conv2D(3, 1, activation = 'softmax', padding='same')(merge9)
    

    model = Model(inputs, conv4)
    opt = SGD(lr=0.01)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    print(model.summary())
    return(model)
model_transfer = semantic_segmentation(input_size=(211,211,8), img_height = 211, img_width = 211, input_channel = 8)
print("[INFO] training network...")
H = model_transfer.fit(
	trainX, trainY, batch_size = BS,
	validation_data = (testX, testY),
	steps_per_epoch=(len(trainX)//BS),
	epochs=EPOCHS, verbose=1) #class_weight=classWeight)

94668760/94668760 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 211, 211, 8)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 14, 14, 2048)      23580480  
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 14, 14, 2048)      0         
                                                                 
 flatten_1 (Flatten)         (None, 401408)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               51380352  
                                                                 
 activation_6 (Activation)   (None, 128)               0         
                                                              

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


None
[INFO] training network...
Epoch 1/100
40/40 [==============================] - 12s 214ms/step - loss: 0.5135 - accuracy: 0.7994 - val_loss: 0.4882 - val_accuracy: 0.8025
Epoch 2/100
40/40 [==============================] - 7s 171ms/step - loss: 0.1770 - accuracy: 0.9656 - val_loss: 0.1788 - val_accuracy: 1.0000
Epoch 3/100
40/40 [==============================] - 7s 170ms/step - loss: 0.0908 - accuracy: 0.9937 - val_loss: 0.0802 - val_accuracy: 1.0000
Epoch 4/100
40/40 [==============================] - 7s 170ms/step - loss: 0.0632 - accuracy: 0.9981 - val_loss: 0.0669 - val_accuracy: 1.0000
Epoch 5/100
40/40 [==============================] - 7s 171ms/step - loss: 0.0459 - accuracy: 0.9984 - val_loss: 0.0313 - val_accuracy: 1.0000
Epoch 6/100
40/40 [==============================] - 7s 170ms/step - loss: 0.0375 - accuracy: 0.9994 - val_loss: 0.0234 - val_accuracy: 1.0000
Epoch 7/100
40/40 [==============================] - 7s 171ms/step - loss: 0.0302 - accuracy: 1.0000 - val_lo

In [17]:
imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2"
imagePaths_list_test = []
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))

imagePaths = "/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 3-4"
for fileName in os.listdir(imagePaths):
	imagePaths_list_test.append(os.path.join(imagePaths, fileName))
test_data = []
test_labels = []
print("length of images_paths_test " + str(imagePaths_list_test))
random.seed(42)
random.shuffle(imagePaths_list_test)

for imagePath in imagePaths_list_test:
	# load the image, pre-process it, and store it in the data list
	print(imagePath)
	var_name = imagePath.split(os.path.sep)[-1]
	if(int(var_name[5:7]) < 10):
		fin_name = var_name[0:5] + '_burn_' + var_name[6]
	else:
		fin_name = var_name[0:5] + '_burn_' + var_name[5:7]
	image = scipy.io.loadmat(imagePath)[fin_name]
	#image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = np.array(image)
	test_data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	test_labels.append(label)
	print(label)
test_data = np.array(test_data, dtype="float") 
#print(labels)
test_labels = np.array(test_labels)
#print("[INFO] data matrix: {:.2f}MB".format(
	#data.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
test_labels = lb.fit_transform(test_labels)
print(test_labels)
results_transfer = model_transfer.evaluate(test_data, test_labels)
print(results_transfer)

length of images_paths_test ['/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_503.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_505.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_507.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_510.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_512.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_516.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_602.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_604.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo Score SFDI - 24 hrs/Histo Score 1-2/pig_606.mat', '/content/drive/MyDrive/Wilson/Raw Dataset by Histo S